In [ ]:
# GPU 성능을 높이기 위한 라이브러리
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.45.2
!pip install fsspec==2024.12.0 gcsfs==2024.12.0
!pip install --upgrade datasets
!pip install -U transformers==4.41.0 trl==0.8.6
!pip install faiss-cpu # 내가 만든 클래스에서 faiss 라이브러리를 사용하기 위해

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sys
# 특정 디렉토리를 모듈 검색 경로에 추가
sys.path.append('/content/drive/MyDrive/healthcare project')

In [ ]:
import pandas as pd
import torch
import joblib
import pandas as pd
import warnings
import textwrap
from sklearn.exceptions import DataConversionWarning
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig, # 메모리 사용량을 줄이기 위해 양자화
    # pipeline
)
from transformers import logging
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from SearchAnswers import SelectReferenceAnswer # 직접 만든 클래스
from sklearn.metrics.pairwise import cosine_similarity

# 내부 경고출력x
warnings.filterwarnings("ignore", category=DataConversionWarning)
logging.set_verbosity_error()

# 학습된 모델 사용을 위한 기본 설정
# 학습된 모델 및 토크나이저 불러오기
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/full_trained_model_instruct", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/full_trained_model_instruct")

# 학습된 분류 모델 불러오기
rf_model = joblib.load('/content/drive/MyDrive/healthcare project/headache_randomforest.pkl')
scaler = joblib.load('/content/drive/MyDrive/healthcare project/headache_scaler.pkl')
pca = joblib.load('/content/drive/MyDrive/healthcare project/headache_pca.pkl')

# 클래스 동작을 위한 정의
answer_df = pd.read_csv("/content/drive/MyDrive/healthcare project/두통 답변.csv", index_col=0)
embed_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
labelencoder = LabelEncoder()

# 객체 생성
select_answers = SelectReferenceAnswer(answer_df, scaler, pca, rf_model, embed_model, labelencoder)
# 벡터 DB 만들기
intention_map, answer_map = select_answers.make_vec_db()


# 프롬프트에 넣기 위한 형식 만드는 함수
def make_ragresult_context(rag_result):
    rag_result_texts = [list(i)[0] for i in rag_result]
    rag_result_context = '\n\n'.join(rag_result_texts)
    return rag_result_context


# 답변 생성 함수
def chat_response(model, tokenizer, user_input):

    rag_result = select_answers.search_answer(user_input, intention_map, answer_map)
    rag_context = make_ragresult_context(rag_result)

    messages = [
        {"role": "system", "content": "너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘. 해당 참고 정보를 참고해서 질문에 답변해줘."},
        {"role": "user", "content": f"""

        [참고 정보]
        {rag_context}

        [질문]
        {user_input}"""}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 생성 종료 조건을 정해줌.
    terminators = [
        tokenizer.eos_token_id, # 이 토큰을 만나면 생성 종료
        tokenizer.convert_tokens_to_ids("<|eot_id|>") # <|eot_id|> 이 토큰 만나도 생성 종료
    ]

    outputs = model.generate(
        input_ids,
        attention_mask=(input_ids != tokenizer.pad_token_id), # 패딩이 아닌 실제 텍스트만 골라서 마스크 생성(마스크: 필요한 정보만 놔두고 나머지 가림)
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
    )
    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer.decode(response, skip_special_tokens=True)

    return textwrap.fill(result, width=60)


if __name__ == "__main__":
    print("🤖 안녕하세요. 의료 AI 챗봇입니다.")

    while True:
        user_input = input("\n🧑 궁금한 부분을 말씀해주세요.\n❓ ")
        if user_input.lower() == 'exit':
            print("❌ 챗봇을 종료합니다.")
            break

        answer = chat_response(model, tokenizer, user_input)
        print("\n🤖 챗봇의 답변:\n", answer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🤖 안녕하세요. 의료 AI 챗봇입니다.

🧑 궁금한 부분을 말씀해주세요.
❓ 오른쪽 머리에서 찌릿하는 두통은 뭔가요?

🤖 챗봇의 답변:
 오른쪽 머리에서 찌릿하는 두통은 일반적으로 편두통의 증상입니다. 편두통은 한쪽 머리에 심한 통증을 일으키는
증상으로, 주로 한쪽 얼굴에 두통이 나타납니다. 이에 따라, 환자는 구토, 빛에 대한 증상, 수면 장애,
어지러움과 같은 다양한 증상도 동반할 수 있습니다. 두통은 발작적으로 나타나며, 주로 밝은 불빛에 대한
민감도가 증가합니다. 이러한 특징을 통해 편두통은 주로 편두통 발작 시에 머리 통증뿐만 아니라 두통, 눈의
통증, 구토, 빛에 대한 민감도 증가 등의 다양한 증상도 동반할 수 있음을 알 수 있습니다. 이러한 증상을
경험하는 경우, 의사와 상담하여 정확한 진단과 치료를 받아야 합니다.

🧑 궁금한 부분을 말씀해주세요.
❓ 두통 예방 방법을 알려주세요

🤖 챗봇의 답변:
 두통 예방을 위해 다양한 방법을 추천합니다. 먼저, 스트레스를 피하는 것이 중요합니다. 스트레스는 두통을
유발할 수 있으므로 가능한 한 피하는 것이 좋습니다. 스트레스 관리 방법을 배우고, 적절한 휴식을 취하는
것이 중요합니다. 또한, 정기적인 신체 활동을 하는 것이 도움이 됩니다. 유산소 운동을 일주일에 최소한
30분 이상 실시하세요. 또한, 올바른 식단을 유지하고, 단백질을 충분히 섭취하는 것이 좋습니다. 건강한
생활습관을 유지하는 것도 중요합니다. 위의 방법을 실천하면 두통을 예방하는 데 도움이 됩니다. 만약 두통이
발생한다면, 즉시 의사와 상담하고 추가적인 예방 조치를 찾아나가세요.

🧑 궁금한 부분을 말씀해주세요.
❓ 스트레스 말고 다른 원인은 뭐가 있나요?

🤖 챗봇의 답변:
 스트레스는 근육의 긴장을 증가시켜 두통을 일으킬 수 있습니다. 또한, 눈의 부담이나 수면 부족이 신경계에
영향을 줌으로써 두통이 발생할 수 있습니다. 이러한 경우 특정 음식이나 음료가 두통을 유발할 수 있는데,
이는 개인에 따라 다양한 반응을 보일 수 있습니다. 